In [30]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [31]:
train_data = pd.read_csv('sales_train_complete_1.csv')
test_data = pd.read_csv('../competitive-data-science-predict-future-sales/test.csv')

In [32]:
obj_dtypes = ['date', 'item_name', 'item_category_name', 'shop_name', 'city']
for col in obj_dtypes:
    train_data[col] = train_data[col].astype('category')
train_data.dtypes


date                        category
date__month                    int64
shop_id                        int64
item_id                        int64
item_price                   float64
item_cnt_day                 float64
item_category_id               int64
item_name                   category
item_category_name          category
shop_name                   category
date__day                      int64
date__week                     int64
date__day_of_month             int64
date__day_of_week              int64
date__week_of_year             int64
date__month_of_year            int64
date__year                     int64
is_weekend                      bool
is_holiday                      bool
item_cnt_month               float64
item_cnt_week                float64
item_cnt_year                float64
dominant_category_vector      object
city                        category
lag_0                        float64
lag_1                        float64
lag_2                        float64
l

In [33]:
test_data['item_cnt_month'] = [0 for i in range(len(test_data.values))]

In [34]:
train_data = train_data.drop(['dominant_category_vector', 'shop_opening_date'], axis = 1)

In [35]:
X_train = train_data.drop(columns=['item_cnt_month'])
y_train = train_data['item_cnt_month']

In [36]:
missing_columns = set(X_train.columns) - set(test_data.columns)
print("Признаки, которых не хватает в тесте:", missing_columns)

Признаки, которых не хватает в тесте: {'lag_8', 'date__day', 'lag_7', 'lag_5', 'lag_3', 'lag_10', 'date__month', 'date__day_of_month', 'item_category_name', 'date__month_of_year', 'item_name', 'date__week', 'lag_4', 'avg_monthly_sales', 'item_cnt_year', 'is_weekend', 'lag_13', 'lag_15', 'lag_11', 'shop_name', 'item_cnt_day', 'store_age', 'city', 'lag_2', 'lag_6', 'item_category_id', 'lag_9', 'is_holiday', 'lag_0', 'item_cnt_week', 'date__week_of_year', 'date', 'date__year', 'date__day_of_week', 'item_price', 'lag_14', 'lag_12', 'lag_1'}


In [37]:
X_test = test_data.copy()

In [38]:
train_data.dtypes

date                   category
date__month               int64
shop_id                   int64
item_id                   int64
item_price              float64
item_cnt_day            float64
item_category_id          int64
item_name              category
item_category_name     category
shop_name              category
date__day                 int64
date__week                int64
date__day_of_month        int64
date__day_of_week         int64
date__week_of_year        int64
date__month_of_year       int64
date__year                int64
is_weekend                 bool
is_holiday                 bool
item_cnt_month          float64
item_cnt_week           float64
item_cnt_year           float64
city                   category
lag_0                   float64
lag_1                   float64
lag_2                   float64
lag_3                   float64
lag_4                   float64
lag_5                   float64
lag_6                   float64
lag_7                   float64
lag_8   

In [39]:
train_data['avg_monthly_sales'] = train_data['avg_monthly_sales'].replace([np.inf, -np.inf], np.nan)
train_data.describe()


,date__month,shop_id,item_id,item_price,item_cnt_day,item_category_id,date__day,date__week,date__day_of_month,date__day_of_week,...,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,store_age,avg_monthly_sales
count,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,1.066138e+06,...,592846.000000,561594.000000,533295.000000,507618.000000,484194.000000,462767.000000,442996.000000,424696.000000,1.066138e+06,1.065228e+06
mean,1.937746e+01,3.037581e+01,9.373190e+03,9.193812e+02,1.138909e+00,2.421833e+01,6.037005e+02,8.579101e+01,1.609792e+01,3.330432e+00,...,2.634539,2.693033,2.749514,2.803707,2.855895,2.905663,2.953740,2.999635,1.594947e+00,5.965159e+00
std,9.160330e+00,1.555579e+01,6.153019e+03,8.943454e+02,4.788792e-01,1.326788e+01,2.787880e+02,3.982778e+01,8.913809e+00,2.004361e+00,...,2.633659,2.676910,2.718178,2.757564,2.794874,2.829872,2.863413,2.895435,7.736602e-01,6.478286e+01
min,0.000000e+00,2.000000e+00,3.000000e+01,2.500000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,2.944498e-02
25%,1.200000e+01,1.900000e+01,4.052000e+03,2.990000e+02,1.000000e+00,1.500000e+01,3.830000e+02,5.400000e+01,8.000000e+00,2.000000e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.863014e-01,2.892889e-01
50%,2.100000e+01,3.100000e+01,7.790000e+03,5.990000e+02,1.000000e+00,2.400000e+01,6.530000e+02,9.300000e+01,1.600000e+01,4.000000e+00,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.695890e+00,8.155889e-01
75%,2.700000e+01,4.400000e+01,1.503100e+04,1.199000e+03,1.000000e+00,3.400000e+01,8.300000e+02,1.180000e+02,2.400000e+01,5.000000e+00,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,2.224658e+00,2.534722e+00
max,3.300000e+01,5.900000e+01,2.216700e+04,5.990000e+03,5.000000e+00,5.500000e+01,1.033000e+03,1.470000e+02,3.100000e+01,6.000000e+00,...,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,2.830137e+00,1.414375e+04


In [40]:
train_data['avg_monthly_sales'].min()

0.0294449822523394

In [41]:
# train_data = train_data.drop(['inf'], axis = 1)

In [42]:
for col in list(train_data.columns):
    print(col)
    train_data[f'{col}'] = train_data[f'{col}'].replace([np.inf, -np.inf], np.nan)


date
date__month
shop_id
item_id
item_price
item_cnt_day
item_category_id
item_name
item_category_name
shop_name
date__day
date__week
date__day_of_month
date__day_of_week
date__week_of_year
date__month_of_year
date__year
is_weekend
is_holiday
item_cnt_month
item_cnt_week
item_cnt_year
city
lag_0
lag_1
lag_2
lag_3
lag_4
lag_5
lag_6
lag_7
lag_8
lag_9
lag_10
lag_11
lag_12
lag_13
lag_14
lag_15
store_age
avg_monthly_sales


In [43]:
model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='rmse',
    early_stopping_rounds=10,
    enable_categorical = True,
)


In [44]:

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)


In [45]:
model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val_split, y_val_split)],
    verbose=True
)


XGBoostError: [15:01:01] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\../data/gradient_index.h:94: Check failed: valid: Input data contains `inf` or a value too large, while `missing` is not set to `inf`

In [ ]:

y_pred = model.predict(X_val_split)
rmse = mean_squared_error(y_val_split, y_pred, squared=False)
print("RMSE:", rmse)


RMSE: 2.7083184959336823


c:\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [44]:
# test_data['item_cnt_month'] = model.predict(X_test)
